In [56]:
from anthropic import Anthropic

client = Anthropic()
MODEL_NAME = "claude-3-sonnet-20240229"

In [57]:
from datetime import datetime

def get_time():
    return datetime.now().strftime("%H:%M:%S")

In [58]:
import re

def calculate(expression):
    # Remove any non-digit or non-operator characters from the expression
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    
    try:
        # Evaluate the expression using the built-in eval() function
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"

tools = [
    {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    },
    {
        "name": "time",
        "description": "Returns the current time in format %H:%M:%S.",
        "input_schema": {
            "type": "object",
            "properties": {
                }
            },
        
    }
]

"""
For claude to use tool he needs a
name of function
description of function
inputs for function
    type
    properties
        (optional)
        input_1
            type of input_1
            description of input_1
        input_2
        ...
    required inputs for function
"""

In [59]:

"""
for a function to acually be called  there needs to be a function that processes the toolname and toolinputs claude has given into an actual python function.
"""
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])
    if tool_name == "time":
        return get_time()

def chat_with_claude(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    message = client.beta.tools.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"Tool Result: {tool_result}")

        response = client.beta.tools.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        response = message

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    print(response.content)
    print(f"\nFinal Response: {final_response}")

    return final_response

In [60]:
chat_with_claude("What is the current time?")
chat_with_claude("What is 2*30-49**2")


User Message: What is the current time?

Initial Response:
Stop Reason: tool_use
Content: [ToolUseBlock(id='toolu_01B3X4CVpjhHwfTKinZamkBB', input={}, name='time', type='tool_use')]

Tool Used: time
Tool Input: {}
Tool Result: 01:44:07
[TextBlock(text='The current time is 01:44:07.', type='text')]

Final Response: The current time is 01:44:07.


'The current time is 01:44:07.'